Здесь тот файл, что mdlj выдает будем приводить к моей csv-шке
---

In [30]:
import random
import os
import time

from numba import jit, njit, vectorize
import numpy as np
import scipy
from numpy.linalg import norm as norm
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from tqdm import tqdm
import gc

from itertools import islice
from collections import defaultdict
import csv

In [31]:
!./md -N 2 -ns 10000 -outf 1 -thermof 100 -dt 0.0005

# NVE MD Simulation of a Lennard - Jones fluid
# L = 1.5874011 rho = 0.5 N = 2 r_cut = 1000000
# Steps number = 10000 seed = 1676570629 dt = 0.0005
# step time PE KE TE drift T
0 0 48 3 51 0 1
100 0.05 14.48266 37.372479 51.855139 0.016767437 12.457493
200 0.1 -0.71227409 52.53857 51.826296 0.016201888 17.512857
300 0.15 0.20246587 51.626409 51.828875 0.01625245 17.208803
400 0.2 -0.80711108 52.639931 51.83282 0.016329809 17.546644
500 0.25 8.659405 43.165666 51.825071 0.016177856 14.388555
600 0.3 11.189982 41.246062 52.436044 0.028157719 13.748687
700 0.35 -0.90042223 53.333796 52.433374 0.028105376 17.777932
800 0.4 0.15804814 52.275269 52.433317 0.028104254 17.42509
900 0.45 -0.9758683 53.409239 52.433371 0.028105306 17.80308
1000 0.5 21.904748 30.538 52.442748 0.028289177 10.179333
1100 0.55 5.5711477 47.567237 53.138385 0.04192911 15.855746
1200 0.6 14.057769 39.114338 53.172108 0.042590346 13.038113
1300 0.65 13.41345 41.121115 54.534565 0.069305191 13.707038
1400 0.7 2.5072032 

In [32]:
N = 2

In [33]:
read_path = f'N{N}.xyz'

In [34]:
# NUMPY VERSION:

def csv_from_xyz(read_path=read_path, use_particles=None, step=1, lines_read_coef=None, velocity_regime=None):
    '''
    создает .csv формат из .vel
    по-сути делает цсв-хи с которыми я работаю из LAMMPS-овского аутпута

    use_particles - количество частиц, которое использовать, то есть сколько из записанных координат использовать (это нормально реализовать супер геморрой)
    recalculate_vels - пересчитать силы
    normalize_vels - нормализовать силы

    step - шаг на количество позиций при чтении (оно в текущей версии очень долго работает с этим параметром)
    lines_read_coef - N * lines_read_coef строчек с координатами считывается - то есть lines_read_coed - количество конфигураций, которое считывается
    '''
    # через решейп к (lines_read, 3) - удаляем с шагом строчки: x = np.delete(x, np.arange(0, x.size, use_particles))

    with open(read_path, 'r+') as read_f:
        all_vels = []
        all_coords = []
        lines_read = 0  # строчки с координатами прочитанные
        for line in (read_f):
            if line[0] != 'L' and len(line) > 2:
                # if lines_read in actual_steps:
                    # if lines_read % (N * step) == 0:    # делаем шаг
                    arr = list(map(lambda x: float(x.strip()), line.split(' ')))
                    arr_coords = (arr[:3])
                    arr_vels = (arr[3:])
                    all_vels.extend(arr_vels)
                    all_coords.extend(arr_coords)
                    lines_read += 1

            if lines_read_coef and lines_read >= lines_read_coef * N:
                break
    all_coords = np.reshape(all_coords, (lines_read // N, 3 * N))
    all_vels = np.reshape(all_vels, (lines_read // N, 3 * N))

    # CFG.N = N
    coords_path = 'coords' + str(N) + '.csv'
    velocities_path = 'velocities' + str(N) + '.csv'

    fieldnames_vels = []
    fieldnames_coords = []
    for i in range(N):
        fieldnames_coords.extend([str(i) + 'x', str(i) + 'y', str(i) + 'z'])

        fieldnames_vels.extend([str(i) + "v_x", str(i) + "v_y", str(i) + "v_z"])

    if not velocity_regime:
        df_coords = pd.DataFrame(all_coords,
                    index=np.arange(len(all_coords)),
                    columns=fieldnames_coords)
        df_coords.index.name = 't'
        df_coords.to_csv(coords_path)  

    df_vels = pd.DataFrame(all_vels,
                index=np.arange(len(all_vels)),
                columns=fieldnames_vels)
    df_vels.index.name = 't'
    df_vels.to_csv(velocities_path)

In [36]:
csv_from_xyz()

In [ ]:
@njit
def pot_energy(r1, r2):
    # Boundary condition realisation:
    r = r1 - r2

    d = norm(r)
    return -4 * (pow(d, -6) - pow(d, -12))

def plot_energy(N=N, path_coords=f"coords{N}.csv", path_vels=f"velocities{N}.csv"):
    E = []
    try:
        coords = pd.read_csv(path_coords).drop(columns=["t"])
        vels = pd.read_csv(path_vels).drop(columns=["t"])
    except:
        coords = pd.read_csv(path_coords)
        vels = pd.read_csv(path_vels)
    
    for row_n in tqdm(range(len(vels))):  # TODO: ЗДЕСЬ СТОИТ vels ПОТОМУ ЧТО У МЕНЯ ПОКА БАГА И В КООРДС НА ОДНУ СТРОЧКУ БОЛЬШЕ ПОЧЕМУ-ТО...
        V = 0
        T = 0
        for i in range(N):
            r_i = np.array(coords.iloc[row_n, (3 * i):(3 * i) + 3])

            for j in range(i + 1, N):
                r_j = np.array(coords.iloc[row_n, (3 * j):(3 * j) + 3])
                # print(r_i, r_j)
                V += pot_energy(r_i, r_j)
        

        T = ((vels.iloc[row_n]) ** 2).sum() / 2
        E.append(T + V)

        # print(
        #     f'{row_n}: K = {T}, V = {V}, E = {T + V}'
        # )

    plt.figure(figsize=(10, 10))
    plt.plot(
        [i for i in range(len(E))], E
    )

    avg_E = np.array(E).mean()

    plt.ylim([-0.1 + avg_E, 0.1 + avg_E])
    # plt.xlabel()